In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
%matplotlib inline
from sklearn.neighbors import KNeighborsClassifier

In [26]:
# creating df with randon ints
df = pd.DataFrame(np.random.randint(10,200, size=(100, 2)), columns=list('AB'))

In [27]:
# creating target variable
df['is_cancer'] = np.random.randint(0,2, df.shape[0])

In [28]:
df['is_cancer'].value_counts()

0    51
1    49
Name: is_cancer, dtype: int64

In [30]:
from sklearn.neighbors import KNeighborsClassifier
neighbors = KNeighborsClassifier(n_neighbors=8)
X = df[['A', 'B']]
Y = df['is_cancer']
neighbors.fit(X,Y)

## Predict for a 24-loudness, 190-second-long song.
print(neighbors.predict([[16, 86]]))
print(neighbors.predict_proba([[16, 86]]))

[0]
[[0.625 0.375]]


At k=5 model predicted positive class or is cancer. At k=8, model predicted negative class correctly. 